In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from repltilian import SwiftREPL
# Create a new Swift REPL process
repl = SwiftREPL()
# Run some code in the Swift REPL
repl.run("var values = [1, 2, 3, 4, 5]")
# Get the value of a variable from Swift to Python
assert repl.vars["values"].get() == [1, 2, 3, 4, 5]
# Create or update variable
repl.vars.set("values", "Array<Int>", list(range(10)))
# Run some more code again
repl.run("""
let offset = -1
let newValues = values.map { $0 * $0 + offset}
""")
assert repl.vars["newValues"].get()[:5] == [-1, 0, 3, 8, 15]
repl.close()

REPL is running !
 24>
values: [Int] = 5 values {
  [0] = 1
  [1] = 2
  [2] = 3
  [3] = 4
  [4] = 5
}
 26>
offset: Int = -1
newValues: [Int] = 10 values {
  [0] = -1
  [1] = 0
  [2] = 3
  [3] = 8
  [4] = 15
  [5] = 24
  [6] = 35
  [7] = 48
  [8] = 63
  [9] = 80
}
 33>


In [3]:
repl = SwiftREPL()
repl.add_reload_file("demo.swift")
repl.run("""
var p1 = Point<Float>(x: 1, y: 2)
var p2 = Point<Float>(x: 2, y: 1)
var p3 = p1 + p2
""", autoreload=True)

assert repl.vars["p3"].get() == {'x': 3, 'y': 3}
repl.close()

REPL is running !
 24>
p1: Point<Float> = {
  x = 1
  y = 2
}
p2: Point<Float> = {
  x = 2
  y = 1
}
p3: Point<Float> = {
  x = 3
  y = 3
}
 111>


In [4]:
from repltilian import SwiftREPL
repl = SwiftREPL()
repl.add_reload_file("demo.swift")
# Run any command to import demo.swift code
repl.run("", autoreload=True)
repl.vars.set("query", "Array<Point<Float>>", [{'x': -1.5, 'y': -1.2}]*100)
repl.vars.set("dataset", "Array<Point<Float>>", [{'x': -1.5, 'y': -1.2}]*5000)

repl.options.output_hide_variables = True
repl.run("""
let newResult = findKNearestNeighbors(query: query, dataset: dataset, k: 10)
""")
assert len(repl.vars["newResult"].get()) == 100
assert len(repl.vars["newResult"].get()[0]["neighbors"]) == 10

# run line profiling on the function findKNearestNeighbors
repl.options.output_hide_variables = True
repl.line_profile(
"""
let newResult = findKNearestNeighbors(query: query, dataset: dataset, k: 10)
""",
function_name="findKNearestNeighbors",
source_path="demo.swift"
)
repl.close()

REPL is running !
 24>
107>
Timer unit: 1 ns
Total time: 0.574 s
Function: findKNearestNeighbors at line 38
Line #      Hits         Time   Per Hit   % Time  Line Contents
     0          1     0.000001  0.000001      0.0%      var results: [SearchResult<T>] = []
     1          1     0.574267  0.574267    100.0%      for queryPoint in query {
     2        100     0.000006  0.000000      0.0%          var distances: [Neighbor<T>] = []
     3        100     0.000002  0.000000      0.0%          // Calculate distances to all dataPoints {
     4        100     0.529691  0.005297     92.2%          for dataPoint in dataset {
     5     500000     0.013678  0.000000      2.4%              let dx = queryPoint.x - dataPoint.x
     6     500000     0.010178  0.000000      1.8%              let dy = queryPoint.y - dataPoint.y
     7     500000     0.012685  0.000000      2.2%              let distance = (dx * dx + dy * dy).squareRoot()
     8     500000     0.043276  0.000000      7.5%        